### 1. 安装HuggingFace 并下载模型到本地

In [2]:
!pip install huggingface-hub -Uqq 
!pip install -U sagemaker

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pytest-astropy 0.8.0 requires pytest-cov>=2.0, which is not installed.
pytest-astropy 0.8.0 requires pytest-filter-subpackage>=0.1, which is not installed.
spyder 4.0.1 requires pyqt5<5.13; python_version >= "3", which is not installed.
spyder 4.0.1 requires pyqtwebengine<5.13; python_version >= "3", which is not installed.
notebook 6.5.5 requires pyzmq<25,>=17, but you have pyzmq 25.1.0 which is incompatible.
sparkmagic 0.20.4 requires nest-asyncio==1.5.5, but you have nest-asyncio 1.5.7 which is incompatible.
spyder 4.0.1 requires jedi==0.14.1, but you have jedi 0.18.2 which is incompatible.
  Using cached sagemaker-2.183.0-py2.py3-none-any.whl
  Obtaining dependency information for urllib3<1.27,>=1.25.4 from https://files.pythonhosted.org/packages/c5/05/c214b32d21c0b465506f95c4f28ccbcba15022e000b043b72b3df77284

In [85]:
from huggingface_hub import snapshot_download
from pathlib import Path
local_model_path = Path("./LLM_bc2_4bit_model")


In [86]:
local_model_path.mkdir(exist_ok=True)
model_name = "baichuan-inc/Baichuan2-13B-Chat-4bits"
commit_hash = "0496919ed797ead809f01c373f1a54251b666690"

In [89]:
snapshot_download(repo_id=model_name, revision=commit_hash, cache_dir=local_model_path)

'LLM_bc2_4bit_model/models--baichuan-inc--Baichuan2-13B-Chat-4bits/snapshots/0496919ed797ead809f01c373f1a54251b666690'

### 2. 把模型拷贝到S3为后续部署做准备

In [90]:
import sagemaker
from sagemaker import image_uris
import boto3
import os
import time
import json

role = sagemaker.get_execution_role()  # execution role for the endpoint
sess = sagemaker.session.Session()  # sagemaker session for interacting with different AWS APIs
bucket = sess.default_bucket()  # bucket to house artifacts

region = sess._region_name
account_id = sess.account_id()

s3_client = boto3.client("s3")
sm_client = boto3.client("sagemaker")
smr_client = boto3.client("sagemaker-runtime")

In [91]:
s3_model_prefix = "LLM-RAG/workshop/LLM_bc2_4bit_stream_model"  # folder where model checkpoint will go
model_snapshot_path = list(local_model_path.glob("**/snapshots/*"))[0]
s3_code_prefix = "LLM-RAG/workshop/LLM_bc2_4bit_stream_deploy_code"
print(f"s3_code_prefix: {s3_code_prefix}")
print(f"model_snapshot_path: {model_snapshot_path}")

s3_code_prefix: LLM-RAG/workshop/LLM_bc2_4bit_stream_deploy_code
model_snapshot_path: LLM_bc2_4bit_model/models--baichuan-inc--Baichuan2-13B-Chat-4bits/snapshots/0496919ed797ead809f01c373f1a54251b666690


In [92]:
!aws s3 cp --recursive {model_snapshot_path} s3://{bucket}/{s3_model_prefix}

upload: LLM_bc2_4bit_model/models--baichuan-inc--Baichuan2-13B-Chat-4bits/snapshots/0496919ed797ead809f01c373f1a54251b666690/handler.py to s3://sagemaker-us-east-2-946277762357/LLM-RAG/workshop/LLM_bc2_4bit_stream_model/handler.py
upload: LLM_bc2_4bit_model/models--baichuan-inc--Baichuan2-13B-Chat-4bits/snapshots/0496919ed797ead809f01c373f1a54251b666690/README.md to s3://sagemaker-us-east-2-946277762357/LLM-RAG/workshop/LLM_bc2_4bit_stream_model/README.md
upload: LLM_bc2_4bit_model/models--baichuan-inc--Baichuan2-13B-Chat-4bits/snapshots/0496919ed797ead809f01c373f1a54251b666690/generation_config.json to s3://sagemaker-us-east-2-946277762357/LLM-RAG/workshop/LLM_bc2_4bit_stream_model/generation_config.json
upload: LLM_bc2_4bit_model/models--baichuan-inc--Baichuan2-13B-Chat-4bits/snapshots/0496919ed797ead809f01c373f1a54251b666690/.gitattributes to s3://sagemaker-us-east-2-946277762357/LLM-RAG/workshop/LLM_bc2_4bit_stream_model/.gitattributes
upload: LLM_bc2_4bit_model/models--baichuan-in

### 3. 模型部署准备（entrypoint脚本，容器镜像，服务配置）

In [95]:
# inference_image_uri = (
#     f"763104351884.dkr.ecr.{region}.amazonaws.com/djl-inference:0.21.0-deepspeed0.8.3-cu117"
# )

inference_image_uri = image_uris.retrieve(
    framework="djl-deepspeed",
    region=sess.boto_session.region_name,
    version="0.23.0"
)
print(f"Image going to be used is ---- > {inference_image_uri}")

Image going to be used is ---- > 763104351884.dkr.ecr.us-east-2.amazonaws.com/djl-inference:0.23.0-deepspeed0.9.5-cu118


In [96]:
!mkdir -p LLM_bc2_4bit_stream_deploy_code

In [97]:
%%writefile LLM_bc2_4bit_stream_deploy_code/model.py
from djl_python import Input, Output
import torch
import logging
import math
import os

from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers.generation import GenerationConfig
STOP_flag = "[DONE]"


def load_model(properties):
    tensor_parallel = properties["tensor_parallel_degree"]
    model_location = properties['model_dir']
    if "model_id" in properties:
        model_location = properties['model_id']
    logging.info(f"Loading model in {model_location}")
    tokenizer = AutoTokenizer.from_pretrained(model_location, use_fast=False, trust_remote_code=True)
    model = AutoModelForCausalLM.from_pretrained(model_location, device_map="auto", torch_dtype=torch.bfloat16, trust_remote_code=True)
    model.generation_config = GenerationConfig.from_pretrained(model_location)

    return model, tokenizer


model = None
tokenizer = None
generator = None


def construct_message(history,prompt):
    message = []
    for question, answer in history:
        message.append({"role":"user","content":question})
        message.append({"role":"assistant","content":answer})
    message.append({"role":"user","content":prompt})
    return message

def stream_items(prompt, history, max_length, top_p, temperature):
    global model, tokenizer
    size = 0
    response = ""
    model.generation_config.max_new_tokens = max_length
    model.generation_config.top_p = top_p
    
    messages = construct_message(history,prompt)
    res_generator = model.chat(tokenizer, messages,stream=True)
    for response in res_generator:
        this_response = response[size:]
        size = len(response)
        stream_buffer = { "outputs":this_response,"finished": False}
        yield stream_buffer
    ## stop
    # yield {"query": prompt, "outputs": STOP_flag, "response": response, "history": [], "finished": True}
    


def handle(inputs: Input):
    global model, tokenizer
    if not model:
        model, tokenizer = load_model(inputs.get_properties())

    if inputs.is_empty():
        return None
    data = inputs.get_as_json()
    
    input_sentences = data["inputs"]
    params = data["parameters"]
    history = data["history"]
    stream = data.get('stream')
    print(f'input prompt:{input_sentences}')    
    outputs = Output()
    if stream:
        outputs.add_property("content-type", "application/jsonlines")
        outputs.add_stream_content(stream_items(input_sentences,history=history,**params))
    else:
        messages = construct_message(history,input_sentences)
        response = model.chat(tokenizer, messages,stream=False)
        result = {"outputs": response, "history" : []}
        outputs.add_as_json(result)
        
    return outputs

Writing LLM_bc2_4bit_stream_deploy_code/model.py


#### Note: option.s3url 需要按照自己的账号进行修改

In [98]:
%%writefile LLM_bc2_4bit_stream_deploy_code/serving.properties
engine=Python
option.tensor_parallel_degree=1
option.enable_streaming=True
option.predict_timeout=240
option.s3url = s3://sagemaker-us-east-2-946277762357/LLM-RAG/workshop/LLM_bc2_4bit_stream_model/

Writing LLM_bc2_4bit_stream_deploy_code/serving.properties


#### 注意: 必须把transformers升级到4.27.1以上，否则会出现 [Issue344](https://github.com/THUDM/ChatGLM-6B/issues/344)

如果是中国区建议添加国内的pip镜像,如下代码所示
```
%%writefile LLM_chatglm_deploy_code/requirements.txt
-i https://pypi.tuna.tsinghua.edu.cn/simple
transformers==4.28.1
```

In [99]:
%%writefile LLM_bc2_4bit_stream_deploy_code/requirements.txt
transformers==4.31.0
accelerate
colorama
bitsandbytes
sentencepiece
transformers_stream_generator
cpm_kernels

Writing LLM_bc2_4bit_stream_deploy_code/requirements.txt


In [100]:
!rm model.tar.gz
!cd LLM_bc2_4bit_stream_deploy_code && rm -rf ".ipynb_checkpoints"
!tar czvf model.tar.gz LLM_bc2_4bit_stream_deploy_code

LLM_bc2_4bit_stream_deploy_code/
LLM_bc2_4bit_stream_deploy_code/model.py
LLM_bc2_4bit_stream_deploy_code/requirements.txt
LLM_bc2_4bit_stream_deploy_code/serving.properties


In [101]:
s3_code_artifact = sess.upload_data("model.tar.gz", bucket, s3_code_prefix)
print(f"S3 Code or Model tar ball uploaded to --- > {s3_code_artifact}")

S3 Code or Model tar ball uploaded to --- > s3://sagemaker-us-east-2-946277762357/LLM-RAG/workshop/LLM_bc2_4bit_stream_deploy_code/model.tar.gz


### 4. 创建模型 & 创建endpoint

In [118]:
from sagemaker.utils import name_from_base
import boto3

model_name = name_from_base(f"bc2-13b-stream") #Note: Need to specify model_name
print(model_name)
print(f"Image going to be used is ---- > {inference_image_uri}")

create_model_response = sm_client.create_model(
    ModelName=model_name,
    ExecutionRoleArn=role,
    PrimaryContainer={
        "Image": inference_image_uri,
        "ModelDataUrl": s3_code_artifact
    },
    
)


model_arn = create_model_response["ModelArn"]

print(f"Created Model: {model_arn}")

bc2-13b-stream-2023-09-08-02-46-28-628
Image going to be used is ---- > 763104351884.dkr.ecr.us-east-2.amazonaws.com/djl-inference:0.23.0-deepspeed0.9.5-cu118
Created Model: arn:aws:sagemaker:us-east-2:946277762357:model/bc2-13b-stream-2023-09-08-02-46-28-628


In [119]:
endpoint_config_name = f"{model_name}-config"
endpoint_name = f"{model_name}-endpoint"

#Note: ml.g4dn.2xlarge 也可以选择
endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "VariantName": "variant1",
            "ModelName": model_name,
            "InstanceType": "ml.g5.2xlarge",
            "InitialInstanceCount": 1,
            # "VolumeSizeInGB" : 400,
            # "ModelDataDownloadTimeoutInSeconds": 2400,
            "ContainerStartupHealthCheckTimeoutInSeconds": 10*60,
        },
    ],
)
endpoint_config_response

{'EndpointConfigArn': 'arn:aws:sagemaker:us-east-2:946277762357:endpoint-config/bc2-13b-stream-2023-09-08-02-46-28-628-config',
 'ResponseMetadata': {'RequestId': '6a90a92a-0a22-44af-9ea5-a47dca17911c',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '6a90a92a-0a22-44af-9ea5-a47dca17911c',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '126',
   'date': 'Fri, 08 Sep 2023 02:46:30 GMT'},
  'RetryAttempts': 0}}

In [120]:
create_endpoint_response = sm_client.create_endpoint(
    EndpointName=f"{endpoint_name}", EndpointConfigName=endpoint_config_name
)
print(f"Created Endpoint: {create_endpoint_response['EndpointArn']}")

Created Endpoint: arn:aws:sagemaker:us-east-2:946277762357:endpoint/bc2-13b-stream-2023-09-08-02-46-28-628-endpoint


#### 持续检测模型部署进度

In [121]:
import time
resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp["EndpointStatus"]
print("Status: " + status)

while status == "Creating":
    time.sleep(60)
    resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
    status = resp["EndpointStatus"]
    print("Status: " + status)

print("Arn: " + resp["EndpointArn"])
print("Status: " + status)

Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: InService
Arn: arn:aws:sagemaker:us-east-2:946277762357:endpoint/bc2-13b-stream-2023-09-08-02-46-28-628-endpoint
Status: InService


### 5. 模型测试

In [28]:
!pip install -U boto3 botocore

  Obtaining dependency information for boto3 from https://files.pythonhosted.org/packages/af/9c/58ca26190a8b89776ec66fdf769d6d0d5287044417b58df4d8225a081153/boto3-1.28.42-py3-none-any.whl.metadata
  Obtaining dependency information for botocore from https://files.pythonhosted.org/packages/c6/1f/334248d06f8cdf397a0aff2e0fad6eb91b731f0c68f85cf93e6ed9835f06/botocore-1.31.42-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 1.6 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 27.4 MB/s eta 0:00:0000:0100:01
DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
  Attempting uninstall: botocore
    Found existing installation: botocore 1.3

In [122]:
%%time
import json
import boto3

smr_client = boto3.client("sagemaker-runtime")

parameters = {
  "max_length": 1024,
  "temperature": 0.1,
  "top_p":0.8
}

CPU times: user 4.21 ms, sys: 0 ns, total: 4.21 ms
Wall time: 4.06 ms


In [123]:
import io


class StreamScanner:
    """
    A helper class for parsing the InvokeEndpointWithResponseStream event stream. 
    
    The output of the model will be in the following format:
    ```
    b'{"outputs": [" a"]}\n'
    b'{"outputs": [" challenging"]}\n'
    b'{"outputs": [" problem"]}\n'
    ...
    ```
    
    While usually each PayloadPart event from the event stream will contain a byte array 
    with a full json, this is not guaranteed and some of the json objects may be split across
    PayloadPart events. For example:
    ```
    {'PayloadPart': {'Bytes': b'{"outputs": '}}
    {'PayloadPart': {'Bytes': b'[" problem"]}\n'}}
    ```
    
    This class accounts for this by concatenating bytes written via the 'write' function
    and then exposing a method which will return lines (ending with a '\n' character) within
    the buffer via the 'readlines' function. It maintains the position of the last read 
    position to ensure that previous bytes are not exposed again. 
    """
    
    def __init__(self):
        self.buff = io.BytesIO()
        self.read_pos = 0
        
    def write(self, content):
        self.buff.seek(0, io.SEEK_END)
        self.buff.write(content)
        
    def readlines(self):
        self.buff.seek(self.read_pos)
        for line in self.buff.readlines():
            if line[-1] != b'\n':
                self.read_pos += len(line)
                yield line[:-1]
                
    def reset(self):
        self.read_pos = 0

## Stream 

In [124]:
import time


# prompts1 = """what's the weather like there?"""
# history = [
#         [
#             "what is the capital city of china?",
#             "Beijing."
#         ]
#     ]
# prompts1 = """AWS Clean Rooms 的FAQ文档有提到 Q: 是否发起者和数据贡献者都会被收费？A: 是单方收费，只有查询的接收方会收费。
# 请问AWS Clean Rooms是多方都会收费吗？
# """
prompts1 = """写一篇500字的科幻小说，背景关于宇宙战争"""
# prompts1 = """床前明月光是谁写的"""
history = []
# endpoint_name = 'bc2-13b-stream-2023-09-07-10-59-15-557-endpoint'

start = time.time()
response_model = smr_client.invoke_endpoint_with_response_stream(
            EndpointName=endpoint_name,
            Body=json.dumps(
            {
                "inputs": prompts1,
                "parameters": parameters,
                "history" : history,
                "stream":True
            }
            ),
            ContentType="application/json",
        )

event_stream = response_model['Body']
scanner = StreamScanner()
for event in event_stream:
    scanner.write(event['PayloadPart']['Bytes'])
    for line in scanner.readlines():
        try:
            resp = json.loads(line)
            print(resp.get("outputs")['outputs'], end='')
        except Exception as e:
            # print(line)
            continue
print (f"time:{time.time()-start} s")

在遥远的未来，宇宙中的各个文明已经发展到了前所未有的高度。在这个时代，科技的发展已经超越了人类的想象，星际旅行已经成为了一种日常现象。然而，随着资源的日益匮乏，宇宙中的各个文明开始为了争夺有限的资源而展开了一场又一场的战争。

在一个名为“诺瓦”的星球上，居住着一群拥有高度智慧的生物。他们被称为诺瓦人，他们的科技水平一个令人难以置信的高度。诺瓦人的社会制度非常先进，他们的政府是由一个名为“议会”的组织来管理的。议会的成员都是由诺瓦人选举产生的，他们负责制定法律、管理国家事务以及对外交事务做出决策。

然而，在这个和平的表面下，诺瓦人却隐藏着一个秘密。他们的星球诺瓦，实际上是一个巨大的武器平台。这个平台的目的是为了保护诺瓦人免受其他宇宙文明的侵略。这个武器平台的力量足以摧毁整个星系，因此，诺瓦人被视为宇宙中最强大的文明之一。

然而，这个秘密最终还是被一个名为“阿尔法”的邪恶组织所发现。阿尔法是一个由宇宙中各种邪恶势力组成的联盟，他们致力于征服整个宇宙。当他们得知诺瓦星球的存在时，他们决定将其视为他们的下一个目标。

阿尔法派出了一支强大的军队，开始对诺瓦星球发起进攻。诺瓦人虽然科技力量，但他们从未经历过如此强大的敌人。在经历了艰苦的战斗后，诺瓦人终于意识到了自己的危险处境。

，诺瓦议会决定启动诺瓦星球上的超级武器。这个武器的力量足以摧毁整个星系，但诺瓦人将永远失去他们的家园。在最后的时刻，诺瓦人向宇宙中的其他文明发出了求救信号。

这个消息很快传遍了整个宇宙，许多文明纷纷派出舰队来支援诺瓦。在这场宇宙规模的战争中，各种先进的武器和战舰在空中交织成一幅壮丽的画卷。然而，这场战争的结局并非一帆风顺。

在经历了漫长的战斗后，诺瓦人和他们的盟友终于击败了阿尔法的军队。然而，诺瓦星球已经被摧毁，诺瓦人也付出了巨大的代价。在这场战争中，许多生命消失了，许多文明受到了重创。

然而，这场战争也让人们认识到了一个事实：在宇宙中，和平并不是永恒的主题。在这个充满竞争和战争的世界里，只有不断地发展和进步，才能确保自己的生存。time:32.3877170085907 s


## None Stream

In [125]:
prompts1 = """AWS Clean Rooms 的FAQ文档有提到 Q: 是否发起者和数据贡献者都会被收费？A: 是单方收费，只有查询的接收方会收费。
请问AWS Clean Rooms是多方都会收费吗？
"""
prompts1 = """写一篇500字的科幻小说，背景关于宇宙战争"""
prompts1 = """床前明月光是谁写的"""
prompts1 = """那里有什么景点"""
history = [
        [
            "中国的首都是哪里",
            "北京."
        ]
    ]
start = time.time()
response_model = smr_client.invoke_endpoint(
            EndpointName=endpoint_name,
            Body=json.dumps(
            {
                "inputs": prompts1,
                "parameters": parameters,
                "history" : [],
            }
            ),
            ContentType="application/json",
        )

resp = response_model['Body'].read()
print (f"\ntime:{time.time()-start} s")
# print(resp.decode('utf8'))
print(json.loads(resp)['outputs'])


time:1.9054903984069824 s
由于您没有提供具体的地点，我无法为您提供确切的景点信息。但是，如果您能告诉我具体的城市或地区，我将很高兴为您提供相关的旅游景点建议。


In [38]:

def construct_message(history,prompt):
    message = []
    for question, answer in history:
        message.append({"role":"user","content":question})
        message.append({"role":"assistant","content":answer})
    message.append({"role":"user","content":prompt})
    return message

In [52]:
prompts1 = """那里有什么景点"""
history = [
        [
            "中国的首都是哪里",
            "北京."
        ]
    ]

In [53]:
messages=construct_message(history,prompts1)

In [54]:
def _parse_messages(messages, split_role="user"):
    system, rounds = "", []
    round = []
    for i, message in enumerate(messages):
        if message["role"] == "system":
            assert i == 0
            system = message["content"]
            continue
        if message["role"] == split_role and round:
            rounds.append(round)
            round = []
        round.append(message)
    if round:
        rounds.append(round)
    return system, rounds

In [81]:
system, rounds = _parse_messages(messages)
history_tokens = []
for round in rounds[::-1]:
    round_tokens = []
    
    for message in round:
        if message["role"] == "user":
            round_tokens.append(1)
        else:
            round_tokens.append(2)
        round_tokens.extend(message["content"])
    history_tokens = round_tokens + history_tokens  # concat left
print(history_tokens)

[1, '中', '国', '的', '首', '都', '是', '哪', '里', 2, '北', '京', '.', 1, '那', '里', '有', '什', '么', '景', '点']


In [73]:
round_tokens

[1, '中', '国', '的', '首', '都', '是', '哪', '里', 2, '北', '京', '.']

In [74]:
rounds[::-1]

[[{'role': 'user', 'content': '那里有什么景点'}],
 [{'role': 'user', 'content': '中国的首都是哪里'},
  {'role': 'assistant', 'content': '北京.'}]]

In [67]:
rounds

[[{'role': 'user', 'content': '中国的首都是哪里'},
  {'role': 'assistant', 'content': '北京.'}],
 [{'role': 'user', 'content': '那里有什么景点'}]]

#### 清除模型Endpoint和config

In [115]:
!aws sagemaker delete-endpoint --endpoint-name {endpoint_name}

In [116]:
!aws sagemaker delete-endpoint-config --endpoint-config-name {endpoint_config_name}

In [117]:
!aws sagemaker delete-model --model-name {model_name}